In [ ]:
"""
Stable Diffusion Image-to-Video Converter
Converts an input image into an animated video using Stable Diffusion img2img pipeline
"""

# Install required packages
%%capture
!pip uninstall diffusers transformers huggingface-hub torch -y
!pip install torch torchvision --upgrade
!pip install diffusers>=0.28.0 transformers>=4.31.0 huggingface-hub>=0.30.0
!pip install pillow opencv-python ipython numpy accelerate imageio imageio-ffmpeg

In [ ]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline, DDIMScheduler
from PIL import Image
import os
import numpy as np
import imageio
from google.colab import files
from IPython.display import display, HTML, Video

# Configuration
MODEL_NAME = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/content/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_stable_model():
    """Loads the Stable Diffusion model with optimized settings"""
    print("🔄 Loading model...")
    try:
        # Use DDIM scheduler for better results
        scheduler = DDIMScheduler.from_pretrained(MODEL_NAME, subfolder="scheduler")

        # Initialize pipeline with fp16 for faster inference
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float16,
            scheduler=scheduler,
            variant="fp16",
            use_safetensors=True
        ).to("cuda")

        # Optimize memory usage
        pipe.enable_attention_slicing()
        pipe.enable_model_cpu_offload()

        print("✅ Model loaded successfully")
        return pipe
    except Exception as e:
        print(f"❌ Error loading model: {str(e)}")
        return None

def create_video(pipe, input_image, prompt, num_frames=24, fps=8):
    """Generates video frames using gradual image transformations"""
    print("🎬 Creating video...")

    frames = []
    current_image = input_image

    for i in range(num_frames):
        print(f"🔨 Generating frame {i+1}/{num_frames}", end="\r")

        # Gradually increase strength for smooth transitions
        strength = min(0.1 + (i/num_frames)*0.7, 0.8)

        result = pipe(
            prompt=prompt,
            image=current_image,
            strength=strength,
            guidance_scale=7.5,
            num_inference_steps=50
        )

        current_image = result.images[0]
        frames.append(np.array(current_image))

    print("\n✅ All frames generated")
    return frames

def save_video(frames, fps=8):
    """Saves frames as an MP4 video file"""
    video_path = os.path.join(OUTPUT_DIR, "output.mp4")
    with imageio.get_writer(video_path, fps=fps) as writer:
        for frame in frames:
            writer.append_data(frame)
    return video_path

def main():
    """Main execution function"""
    print("📤 Please upload an image (preferably 512x512 pixels)...")
    uploaded = files.upload()
    if not uploaded:
        print("❌ No image uploaded")
        return

    image_file = next(iter(uploaded))
    prompt = input("✏️ Enter transformation prompt (in English): ") or "a beautiful animation"

    try:
        # Process input image
        img = Image.open(image_file).convert("RGB")
        if img.size != (512, 512):
            print("⚠️ Resizing image to 512x512")
            img = img.resize((512, 512))

        # Load model
        pipe = load_stable_model()
        if pipe is None:
            return

        # Generate and save video
        frames = create_video(pipe, img, prompt)
        video_path = save_video(frames)

        # Display results
        display(HTML(f"""
        <div style="text-align:center; margin:20px">
            <h3 style="color:green">✅ Video ready!</h3>
            <video width="512" controls style="margin:20px; border:2px solid #4CAF50; border-radius:8px;">
                <source src="{video_path}" type="video/mp4">
                Your browser doesn't support video tag.
            </video>
            <div style="margin-top:20px;">
                <a href="{video_path}" download="generated_video.mp4" style="
                    display:inline-block;
                    padding:12px 24px;
                    background:#4CAF50;
                    color:white;
                    text-decoration:none;
                    border-radius:5px;
                    margin:10px;
                    font-weight:bold;
                    font-size:16px;">
                    Download Video
                </a>
            </div>
        </div>
        """))

        display(Video(video_path, embed=True))

    except Exception as e:
        print(f"❌ Processing error: {str(e)}")
    finally:
        if 'pipe' in locals():
            del pipe
        torch.cuda.empty_cache()
        print("GPU memory cleared")

if __name__ == "__main__":
    print("""
    🎥 Stable Diffusion Image-to-Video Converter
    -------------------------------------------
    Transform your images into animated videos using AI
    """)
    main()
    print("🏁 Process completed successfully")